#### RDD Operations Exercise customer and salestxn data set with broadcase variable

We have customer details in a file - customers.tsv and sales transactions details in another file - salestxn.tsv.
Sales transactions contain only customer id but not name. Also the transactions may have multiple entries for a particular item if customer made separate transactions. We have to generate output that contain customer id, customer name along with the product name, product id, price, total quantity ordered and total amount paid.

Since customer file is smaller (with 1244 records) and sales transactions file is larger (172198 records) we will put customer details in a variable and broadcast it across the cluster in our code. We will use this broadcast variable to add the customer id and name to the sales transactions.

In the application flow we also need to aggregate the sales transactions of the same product in cases where a customer ordered as multiple transactions.

Import findspark with spark installation path and initialize 

In [1]:
import findspark
findspark.init('/usr/local/spark')

Import PySpark and create SparkContext

In [2]:
import pyspark
sc = pyspark.SparkContext(appName='RDD Operations Broadcast Variable Example')

Read the customers data input file to create an input RDD

In [3]:
custRDD1=sc.textFile("customers.tsv")

Display count and a sample of the RDD elements for debugging purposes. The RDD elements are the lines of type string from the input file.

In [4]:
custRDD1.count()

1244

In [5]:
custRDD1.take(3)

['11039\tMary Torres\tCaguas\tPR\t725',
 '5623\tJose Haley\tColumbus\tOH\t43207',
 '5829\tMary Smith\tHouston\tTX\t77015']

In [6]:
type(custRDD1.first())

str

###### We need to transform the RDD elements into key,value pairs with the required key and value.

The function for custRDD:
 - Splits the input string at tabs.
 - Uses customer id as the key and customer name as the value
 - Then returns the string constructed as pair RDD

In [7]:
def make_cust_pairRDD(str):
    line = str.split('\t')
    return (line[0], line[1])

Use map lambda function to transform each element of RDD1 to key,value pair using the above function. 

In [8]:
custRDD2=custRDD1.map(lambda line: make_cust_pairRDD(line))

Normally persist is done when we have to do several transformations on an RDD. It will avoid recomputing the RDD from the beginning for each transformation.

In [9]:
custRDD2.persist()

PythonRDD[5] at RDD at PythonRDD.scala:49

In [10]:
custRDD2.take(5)

[('11039', 'Mary Torres'),
 ('5623', 'Jose Haley'),
 ('5829', 'Mary Smith'),
 ('6336', 'Richard Maddox'),
 ('1708', 'Margaret Booth')]

In [11]:
type(custRDD2.first())

tuple

In [12]:
custDict=custRDD2.collectAsMap() #collect key value pairs and return a dictionary

The RDD operation collectAsMap returns the pairRDD as a dictionary (referred to as Map in other programming languages).

In [13]:
type(custDict)

dict

In [14]:
for k, v in custDict.items():
    print(k, v)

8638 Daniel Wood
9001 Mary Smith
8260 Teresa Berg
11021 Brenda Shaw
1682 Mary Hall
2705 Mary Smith
8973 Mary Smith
8641 David Smith
3281 Teresa Werner
11463 Mary Smith
8530 William Smith
11684 Denise Smith
1215 Joe Hart
576 Arthur Fisher
9380 Ryan Carson
2928 Sarah Larsen
3077 Albert Rush
11819 Jerry Vincent
12334 Michael Robertson
10377 Anna Smith
11404 Helen Cook
8686 Dorothy Higgins
7786 Mary Price
4202 Kimberly Griffin
8226 Mary Ritter
3242 Mary Garcia
5435 Mary Smith
4255 Brittany Smith
8243 Gary Walker
1221 Mary Smith
5161 Mary Zavala
9919 Mary Daniel
7074 Mary Patton
10626 Emma Anderson
9326 Linda Hess
4737 Mary Mendoza
7408 Mary Holmes
9570 William Sampson
6654 Mary Simmons
8157 Mary Marshall
12118 Mary Zimmerman
6196 Jennifer Stewart
872 Keith Barnes
1653 Brian Gates
9317 Mary Smith
9778 Kathy Maddox
967 Madison Smith
8348 Mary Maddox
5500 Diane Smith
5120 Mary Smith
3846 Ronald Lewis
12271 Mary Small
6485 Carolyn Sheppard
1119 Bryan Ramos
7317 Mary Smith
7458 Philip Williams


7965 Mary Hunter
651 Mary Smith
7500 Mary Smith
1887 Katherine Jenkins
9458 Mary Carlson
11877 Jeffrey Krueger
11917 Matthew Brooks
1598 Mary Green
7919 Brittany Kent
6529 Aaron Smith
4327 Sarah Greene
8784 Jonathan Smith
10895 Mary Pierce
821 Daniel Clayton
10472 Margaret Cook
6339 Mary Bush
2245 Megan Johnson
10647 Amy Johnson
5169 Tiffany Berger
409 Kevin Smith
12431 Mary Rios
7544 Barbara Smith
4181 Aaron Hudson
5577 Mary Smith
10371 Joseph Smith
11114 Mary Owens
2250 Amanda Smith
4768 Amanda Hays
1557 Nicole Bolton
714 Mark Beltran
11072 Nathan Anthony
11693 Mary Garrett
7415 Mary Smith
6930 Paul Morton
9440 Mary Leonard
1662 George Smith
1677 Virginia Robles
1247 Robert Montgomery
5960 Margaret Smith
5479 Mary Smith
611 Lisa Cross
6174 Carl Bright
4064 Mildred Smith
8379 Martha Kelly
3572 Mary Bennett
1158 Emily Price
10225 Diane English
1369 Mary Walker
2082 Megan Holland
5755 Marie Shah
3519 Nicholas Ruiz
2161 Mary Mercer
5788 Mary Barrera
3932 Evelyn Smith
5009 Aaron Smith
845

8364 Michael Smith
7198 Mary Day
5792 Mary Smith
2321 Mary Patel
3357 Mary Smith
3865 Jessica Hunter
9338 James Davis
8943 Brian Yates
8456 Mary Smith
8034 Mary Smith
8387 Richard Fuentes
12346 Mary Smith
4156 Mary Smith
10547 Amy Ruiz
11681 Roy Jackson
9461 Mary Hoover
10525 Mary Lopez
5301 Mary Ward
1076 Tyler Thompson
5229 Thomas Butler
11857 Judy Wall
8053 Mary Dennis
10163 Theresa Smith
11537 Lori Phelps
5603 David Smith
1684 Anna Smith
5864 Mary Gentry
268 Mary Miller
5952 Jacob Smith
2527 Heather Smith
5973 Michael Smith
8178 Mary Barnes
4220 Mary Morgan
8643 Mary Valenzuela
5253 Susan Watkins
189 Angela Peterson
11916 Mary Smith
803 James Hartman
6285 Mary Garcia
2891 Victoria Pitts
1636 Julie Petersen
1745 Mary Smith
10413 Mary Smith
4604 Mary Pearson
9708 Cynthia Smith


In [15]:
custDict['6342']

'Kimberly Blair'

In [16]:
custDict['11599']

KeyError: '11599'

Since some customer details are missing in the customers files we better use .get method to get the customer name for a given customer id.

In [17]:
custDict.get('6342')

'Kimberly Blair'

In [18]:
custDict.get('11599')

We will define a broadcast variable - custDictBroadcast and make our custDict as the broadcast variable in our application.

In [19]:
custDictBroadcast=sc.broadcast(custDict)

In [20]:
type(custDictBroadcast)

pyspark.broadcast.Broadcast

In [21]:
custDictBroadcast.value

{'9001': 'Mary Smith',
 '8260': 'Teresa Berg',
 '9158': 'Mary Bender',
 '1682': 'Mary Hall',
 '2705': 'Mary Smith',
 '8973': 'Mary Smith',
 '8641': 'David Smith',
 '3281': 'Teresa Werner',
 '8530': 'William Smith',
 '11684': 'Denise Smith',
 '1215': 'Joe Hart',
 '576': 'Arthur Fisher',
 '9380': 'Ryan Carson',
 '2928': 'Sarah Larsen',
 '3077': 'Albert Rush',
 '3627': 'Mary Smith',
 '12334': 'Michael Robertson',
 '10377': 'Anna Smith',
 '11404': 'Helen Cook',
 '8686': 'Dorothy Higgins',
 '7786': 'Mary Price',
 '8226': 'Mary Ritter',
 '6493': 'Mary Best',
 '5435': 'Mary Smith',
 '5960': 'Margaret Smith',
 '8243': 'Gary Walker',
 '1221': 'Mary Smith',
 '5161': 'Mary Zavala',
 '8211': 'Ronald Mcneil',
 '7074': 'Mary Patton',
 '10626': 'Emma Anderson',
 '9326': 'Linda Hess',
 '4737': 'Mary Mendoza',
 '7408': 'Mary Holmes',
 '9570': 'William Sampson',
 '1835': 'Mary Smith',
 '6654': 'Mary Simmons',
 '8157': 'Mary Marshall',
 '12118': 'Mary Zimmerman',
 '6196': 'Jennifer Stewart',
 '5797': 'Ro

In [22]:
type(custDictBroadcast.value)

dict

In [23]:
custDictBroadcast.value['6342']

'Kimberly Blair'

In [24]:
custDictBroadcast.value['11599']

KeyError: '11599'

Since some customer details are missing in the customers files we better use .get method to get the customer name for a given customer id. the .get method returns none if the key does not exist instead of giving an error.

In [25]:
custDictBroadcast.value.get('6342')

'Kimberly Blair'

In [26]:
custDictBroadcast.value.get('11599') #no error

In [27]:
for k, v in custDictBroadcast.value.items():
    print(k, v)

9001 Mary Smith
8260 Teresa Berg
9158 Mary Bender
1682 Mary Hall
2705 Mary Smith
8973 Mary Smith
8641 David Smith
3281 Teresa Werner
8530 William Smith
11684 Denise Smith
1215 Joe Hart
576 Arthur Fisher
9380 Ryan Carson
2928 Sarah Larsen
3077 Albert Rush
3627 Mary Smith
12334 Michael Robertson
10377 Anna Smith
11404 Helen Cook
8686 Dorothy Higgins
7786 Mary Price
8226 Mary Ritter
6493 Mary Best
5435 Mary Smith
5960 Margaret Smith
8243 Gary Walker
1221 Mary Smith
5161 Mary Zavala
8211 Ronald Mcneil
7074 Mary Patton
10626 Emma Anderson
9326 Linda Hess
4737 Mary Mendoza
7408 Mary Holmes
9570 William Sampson
1835 Mary Smith
6654 Mary Simmons
8157 Mary Marshall
12118 Mary Zimmerman
6196 Jennifer Stewart
5797 Roy Smith
2459 Robert Lyons
4319 Melissa Smith
9778 Kathy Maddox
8348 Mary Maddox
5120 Mary Smith
3846 Ronald Lewis
12271 Mary Small
6485 Carolyn Sheppard
7458 Philip Williams
3132 Mary Gutierrez
11444 Kathleen Patrick
1119 Bryan Ramos
3828 Janice Carpenter
580 Kimberly Lee
11025 Mary S

10005 Zachary Day
11314 Ashley Mclaughlin
7491 Sharon Dominguez
9818 Mary Avila
8295 Margaret Lynch
3929 Amanda Levy
12417 Mary Smith
4558 Jennifer Smith
11261 Robert Smith
10695 Andrew Smith
10832 Mary Khan
11799 Mary Tate
1283 Ashley White
9624 Ann Smith
3339 Mary Payne
3621 Mary West
2636 Christian Smith
4777 David Smith
330 Mary Smith
10528 Mary Smith
9815 Cheryl Simmons
4843 Mary Sandoval
5234 Shirley Humphrey
131 Brenda Cline
8695 Larry Thomas
2420 Mary Smith
6323 Gerald Roberts
5013 Mary Peters
7176 Mary Hudson
5397 Mary Nelson
9957 Charles Bennett
24 Mary Smith
2875 Mary Roberts
3156 Kathleen Smith
900 George Smith
9139 Mary Galvan
5406 Mary Whitehead
9031 Mary Brown
2571 Zachary Beasley
547 Kathleen Contreras
12400 Mary Smith
9135 Mary Johnson
12035 William Mcdaniel
11589 Mary Anderson
11128 Timothy Lyons
11385 Mary Stewart
4627 Eric Lang
409 Kevin Smith
7794 Jennifer Compton
5759 Mary Brown
9418 Debra Snow
5119 Brenda Miller
11845 Mary Weaver
7440 James French
11363 Frank Pau

4210 Matthew Roach
12108 Gloria Stephens
9934 Kathleen Smith
2504 Mary Smith
4929 Janet Harvey
9674 Sean Smith
11526 Dorothy Crawford
8346 Mary Wright
1708 Margaret Booth
8630 Jeremy Smith
152 Mary Smith
2087 Mary Burton
10616 Mary Little
2990 Michael Smith
371 Adam Marquez
1334 Roger Dunlap
8201 Lauren Smith
10303 Shawn Smith
5893 Mary Young
11173 Kathleen Mayer
6532 Mary Dixon
8731 Donna Allen
12336 Joe Smith
8591 Mary Smith
1785 Mary Smith
655 Joseph Dougherty
11181 Mary Hernandez
7622 Mary Hodges
7904 Mary Smith
10612 Anna Smith
9657 Roy Williams
10054 Zachary Hunt
5308 Mary Smith
8375 John Smith
9600 Nicholas Sanders
1322 Aaron Williamson
8364 Michael Smith
7198 Mary Day
5792 Mary Smith
2023 Mary Fox
3357 Mary Smith
3865 Jessica Hunter
9338 James Davis
8943 Brian Yates
22 Joseph Smith
6483 Mary Smith
2471 Olivia Smith
1824 Julia Tucker
4156 Mary Smith
6631 Mary Morrow
10547 Amy Ruiz
9461 Mary Hoover
4041 Mary Santiago
7091 Elizabeth Wells
10559 Mary Jones
5229 Thomas Butler
7583 M

Read the salestxns data input file to create another input RDD

In [28]:
salesRDD1=sc.textFile("salestxns.tsv")

Display count and a sample of the RDD elements for debugging purposes. The RDD elements are the lines of type string from the input file.

In [29]:
salesRDD1.count()

172198

In [30]:
salesRDD1.take(3)

["1\t43\tCamping & Hiking\t957\tDiamondback Women's Serene Classic Comfort Bi\t299.98\t1\t11599",
 '2\t48\tWater Sports\t1073\tPelican Sunstream 100 Kayak\t199.99\t1\t256',
 "3\t24\tWomen's Apparel\t502\tNike Men's Dri-FIT Victory Golf Polo\t50\t5\t256"]

In [31]:
type(salesRDD1.first())

str

###### We need to transform the RDD elements into key,value pairs with the required key and value.

The function for salesRDD:
 - Splits the input string at tabs.
 - Uses a tuple containing customer id, product id, product name and product price as the key and product quantity as the value
 - Then returns the string constructed as pair RDD
 - Later in the code we will use reduceByKey to sum up the quantities of same product bought by same customer in separate multiple transactions
 

In [32]:
def make_sales_pairRDD(str):
    line = str.split('\t')
    return ((line[7],line[3],line[4],line[5]), int(line[6])) #((details),qty) tuple inside tuple

In [33]:
salesRDD2=salesRDD1.map(lambda line: make_sales_pairRDD(line))

Display the RDD2 elements for debug purposes.

Since sales RDD is more than 100 times larger than customer RDD we can increase the parallelism  by increasing the partitions using partitionBy.
PySpark uses a default partitioner in this case.

In [34]:
salesRDD2.partitionBy(10).persist()

MapPartitionsRDD[16] at mapPartitions at PythonRDD.scala:443

In [35]:
salesRDD2.count()

172198

In [36]:
salesRDD2.take(3)

[(('11599', '957', "Diamondback Women's Serene Classic Comfort Bi", '299.98'),
  1),
 (('256', '1073', 'Pelican Sunstream 100 Kayak', '199.99'), 1),
 (('256', '502', "Nike Men's Dri-FIT Victory Golf Polo", '50'), 5)]

In [38]:
type(salesRDD2.first())

tuple

In [39]:
salesRDD3 = salesRDD2.reduceByKey(lambda va,vb: va+vb)

In [40]:
salesRDD3.count()

92875

In [41]:
salesRDD3.take(3)

[(('10223', '502', "Nike Men's Dri-FIT Victory Golf Polo", '50'), 5),
 (('11767', '957', "Diamondback Women's Serene Classic Comfort Bi", '299.98'),
  1),
 (('8136', '306', 'Polar FT4 Heart Rate Monitor', '89.99'), 3)]

Map the key and value in each element of salesRDD3 so that we have customer id, product id, product name, price, total quantity purchased and lastly total amount paid (price * total qty).
Once that is done we can use the broadcast variable to get the customer name as required for the final format mentioned in the problem statement.

In [42]:
salesRDD4=salesRDD3.map(lambda fld : (fld[0][0],fld[0][1],fld[0][2],float(fld[0][3]),fld[1], 
                                      round(float(fld[0][3])*fld[1],2)))

In [43]:
salesRDD4.take(5)

[('10223', '502', "Nike Men's Dri-FIT Victory Golf Polo", 50.0, 5, 250.0),
 ('11767',
  '957',
  "Diamondback Women's Serene Classic Comfort Bi",
  299.98,
  1,
  299.98),
 ('8136', '306', 'Polar FT4 Heart Rate Monitor', 89.99, 3, 269.97),
 ('2222', '191', "Nike Men's Free 5.0+ Running Shoe", 99.99, 3, 299.97),
 ('2636', '365', 'Perfect Fitness Perfect Rip Deck', 59.99, 5, 299.95)]

In [44]:
type(salesRDD3.first())

tuple

In [45]:
salesRDD5=salesRDD4.map(lambda fld : (fld[0],custDictBroadcast.value.get(fld[0]),fld[1],fld[2],fld[3],fld[4],fld[5]))

In [46]:
for line in salesRDD5.take(15):
    print(line)

('10223', None, '502', "Nike Men's Dri-FIT Victory Golf Polo", 50.0, 5, 250.0)
('11767', None, '957', "Diamondback Women's Serene Classic Comfort Bi", 299.98, 1, 299.98)
('8136', None, '306', 'Polar FT4 Heart Rate Monitor', 89.99, 3, 269.97)
('2222', None, '191', "Nike Men's Free 5.0+ Running Shoe", 99.99, 3, 299.97)
('2636', 'Christian Smith', '365', 'Perfect Fitness Perfect Rip Deck', 59.99, 5, 299.95)
('20', None, '365', 'Perfect Fitness Perfect Rip Deck', 59.99, 14, 839.86)
('8808', None, '1073', 'Pelican Sunstream 100 Kayak', 199.99, 5, 999.95)
('2883', None, '957', "Diamondback Women's Serene Classic Comfort Bi", 299.98, 2, 599.96)
('4506', None, '1073', 'Pelican Sunstream 100 Kayak', 199.99, 2, 399.98)
('9989', None, '886', 'Team Golf San Francisco Giants Putter Grip', 24.99, 2, 49.98)
('263', None, '135', 'Nike Dri-FIT Crew Sock 6 Pack', 22.0, 4, 88.0)
('744', None, '365', 'Perfect Fitness Perfect Rip Deck', 59.99, 24, 1439.76)
('7602', 'Brian Mitchell', '957', "Diamondback Wom

In [47]:
salesRDD5.saveAsTextFile('cust_sales55')

The above code has several print statements for illustration and for debug purposes. Also, some of the code can be improved. For example we do not need salesRDD5. We can get the required result without creating it thus reducing one transformation. Figure out any other such improvements and rewrite the code.

In [48]:
sc.stop()

Also figure out how you can get the same output when you use join the two RDDs (customers and salestxns)